In [1]:
from azureml.tensorboard import Tensorboard
from azureml.core import Experiment, Environment, Workspace, Datastore, Dataset, Model, ScriptRunConfig
import os
import glob
# get the current workspace
ws = Workspace.from_config()

In [21]:
# access our registered data share containing image data in this workspace
datastore = Datastore.get(workspace = ws, datastore_name = 'wetlanddatastore')

train_path = (datastore, 'data/training/')
eval_path = (datastore, 'data/eval/')

test_path = (datastore, 'data/predict/test_aoi_David1_DE')

train_dataset = Dataset.File.from_files(path = [train_path])
eval_dataset = Dataset.File.from_files(path = [eval_path])
test_dataset = Dataset.File.from_files(path = [test_path])

In [26]:
run_id = 'wetland-unet_1623274083_79c16e68'

run = ws.get_run(run_id)

run.get_file_names()


['azureml-logs/55_azureml-execution-tvmps_125cd859023dbee460a80b92b097acec6f6261583a1dd36cc3da010b21063da8_d.txt',
 'azureml-logs/65_job_prep-tvmps_125cd859023dbee460a80b92b097acec6f6261583a1dd36cc3da010b21063da8_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_125cd859023dbee460a80b92b097acec6f6261583a1dd36cc3da010b21063da8_d.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'logs/azureml/87_azureml.log',
 'logs/azureml/dataprep/backgroundProcess.log',
 'logs/azureml/dataprep/backgroundProcess_Telemetry.log',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log',
 'logs/azureml/sidecar/tvmps_125cd859023dbee460a80b92b097acec6f6261583a1dd36cc3da010b21063da8_d/all.log',
 'logs/azureml/sidecar/tvmps_125cd859023dbee460a80b92b097acec6f6261583a1dd36cc3da010b21063da8_d/task.enter_contexts.log',
 'logs/azureml/sidecar/tvmps_125cd859023dbee460a80b92b097acec6f6261583a1dd36cc3da010b21063da8_d/task.exit_contexts.log',


In [39]:
run.download_files(prefix = 'logs', output_directory = 'logs')

In [22]:
model = run.register_model(model_name='wetland-unet-test',
                           tags=run.tags,
                           description = 'UNET model delineating wetlands in Delaware, Minnesota, and New York',
                           model_path='outputs/',
                           model_framework = 'Tensorflow',
                           model_framework_version= '2.0',
                           datasets = [('training', train_dataset), ('evaluation', eval_dataset), ('testing', test_dataset)])
print(model.name, model.id, model.version, sep='\t')

wetland-unet-test	wetland-unet-test:1	1


In [2]:
## NOTE: the log files written during train_wetland were successfully read by tensorboard in 
## my mevans@defenders.org env. Tensorboard likely not working here due to some kind of permission issue

# The TensorBoard constructor takes an array of runs, so be sure and pass it in as a single-element array here
tb = Tensorboard(runs = [], local_root = '/home/azureuser/cloudfiles/code/Users/mevans/basic_logs', port = 6006)
# tb = Tensorboard(runs = [run], port = 6006)
# If successful, start() returns a string with the URI of the instance.
tb.start()

https://wetlandsbasiccpu-6006.eastus.instances.azureml.ms


'https://wetlandsbasiccpu-6006.eastus.instances.azureml.ms'

In [49]:
tb.stop()
Users/mevans/basic_logs